# Set up, data, dataframe creation

In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import os

#Set working directory 
os.chdir('/Users/Davide/Documents/University/RA')
os.getcwd()


'/Users/Davide/Documents/University/RA'

In [2]:
headlines_df = pd.read_csv('express.csv', index_col = 'date')

In [3]:
headlines_df

,headline,newspaper
date,,
1-3-2015,Prince Andrew 'frozen out by Charles over dama...,Express
1-3-2015,EXCLUSIVE: Migrants to put Britain's populatio...,Express
1-3-2015,EXCLUSIVE: Jihadi John exposed by web error: K...,Express
1-3-2015,Missing Becky Watts: Two arrested as family ad...,Express
1-3-2015,We can all benefit from a positive approach to...,Express
...,...,...
31-3-2017,Man United player: I was worried after manager...,Express
31-3-2017,Mark Lawrenson: My biggest worry about Liverpool,Express
31-3-2017,La Liga ace reveals messages received from Man...,Express


# Classification

In [4]:
british_keywords = ["brexit", "european union", " eu ", "british", "british identity", "british passport", "british culture", "british heritage", "british goods", "british products","british manufacturing", "made in britain"]
climate_keywords = ["climate change", "paris agreement", "environment", "global warming", "unfccc"]

headlines_df['classification'] = ''
classification_lst = []


def headline_classifier(string, keyword_lst_class_1, keyword_lst_class_2, name_class_1 = 'class_1', name_class_2 = 'class_2'):
    '''Classifies a list of strings according to keyword lists for class_1 and class_2'''
    string = string.lower()
    count_1 = 0
    count_2 = 0
    for keyword in keyword_lst_class_1:
        if keyword in string:
            count_1 =+ 1
    for keyword in keyword_lst_class_2:

        if keyword in string:
            count_2 =+ 1
    
    if count_1 > 0 and count_2 > 0:
        return 'both'
    elif count_1 > 0:
        return name_class_1
    elif count_2 > 0:
        return name_class_2
    else:
        return 'none'

In [6]:
n_row, n_col = headlines_df.shape
headlines_lst = list(headlines_df['headline'].values)

#Headline classification
classification_lst = []
for i in range(n_row):
    headline_i = str(headlines_lst[i]) #Include this becasue of potential nan values
    classification_lst.append(headline_classifier(headline_i, british_keywords, climate_keywords, name_class_1 = 'Brexit', name_class_2 = 'Climate'))

headlines_df['classification'] = classification_lst
headlines_df.head(50)
headlines_df.classification.value_counts()

none       207415
Brexit      11789
Climate       207
both           11
Name: classification, dtype: int64

In [232]:
#Testing
#To do find more elegant way to deal with eu than " eu "
test_lst = []

test_headline = headlines_df.iloc[43]['headline']     
test_lst.append(headline_classifier(test_headline, british_keywords, climate_keywords, name_class_1 = 'Brexit', name_class_2 = 'Climate'))              
test_lst

test_lst

['none']

# Sentiment Analysis

In [12]:
#Empty list to add the polarity score
polarity_lst = []
subjectivity_lst = []

#Headline sentiment
for i in range(n_row):
    headline_i = str(headlines_lst[i]) #Include this becasue of potential nan values
    blob_headline_i = TextBlob(headline_i) #transforming string into textblob
    polarity_lst.append(blob_headline_i.sentiment.polarity)
    subjectivity_lst.append(blob_headline_i.sentiment.subjectivity)

#Adding polarity and subjectivity scores to the headlines dataframe
headlines_df['polarity'] = polarity_lst
headlines_df['subjectivity'] = subjectivity_lst
headlines_df

,headline,newspaper,classification,polarity,subjectivity
date,,,,,
1-3-2015,'We must monitor Islamic student societies wit...,Express,Brexit,0.000000,0.000000
2-3-2015,Now EU targets halogen bulbs: Brussels could B...,Express,Brexit,0.200000,0.350000
2-3-2015,In bed with Brook: British model Kelly shows o...,Express,Brexit,0.000000,0.000000
2-3-2015,Tory MP says Britain would be better off witho...,Express,Brexit,0.500000,0.500000
2-3-2015,Two out of three British laws were made in Bru...,Express,Brexit,0.000000,0.000000
...,...,...,...,...,...
31-3-2017,Paul Nuttall hails Britain’s ‘great future’ & ...,Express,Brexit,0.400000,0.437500
31-3-2017,Hitler 'stopped advancing on British troops at...,Express,Brexit,0.000000,0.000000
31-3-2017,Chris Ashton opens up about possibility of get...,Express,Brexit,0.000000,0.000000


In [25]:
#Creating dataframe for each class
brexit_df = headlines_df[headlines_df.classification == 'Brexit']
climate_df = headlines_df[headlines_df.classification == 'Climate']
both_df = headlines_df[headlines_df.classification == 'both']

#
results_df = pd.DataFrame(index = ['polarity', 'subjectivity'], columns = ['Brexit', 'Climate', 'Both'])
results_df['Brexit'] = brexit_df.mean(axis = 0, numeric_only = True)
results_df['Climate'] = climate_df.mean(axis = 0, numeric_only = True)
results_df['Both'] = both_df.mean(axis = 0, numeric_only = True)

results_df

,Brexit,Climate,Both
polarity,0.026300,0.012187,-0.139876
subjectivity,0.246679,0.282840,0.339050
